# Сultural heritage in Moscow
Мой знакомый спросил меня - как по твоему, какие районы города наиболее интересные с точки зрения объектов культурного наследия и при этом имеющин не самые высокие цены на квартиры?
В данном проекте мы попытаемся ответить на этот вопрос: оценить и классифицировать объекты культурного наследия (далее ОКН) в г.Москве, Россия. Определим наиболее интересные районы города с точки зрения оценок пользователей сервиса Foursquare и соотнесем эти данные со средней стоимостью квартир в том или  инном районе.
### Современная ситуация в Москве
На территории города Москвы расположено более 7 500 объектов, находящихся под государственной охраной.
Наибольшее число Объектов сконцентрировано в Центральном административном округе и самые высокие цены на недвижемость.
### А знаете ли вы, что?
На территории города Москвы расположено 3 объекта культурного наследия, включенных в Список всемирного культурного и природного наследия ЮНЕСКО:
* Ансамбль Кремля и Красная площадь
* Церковь Вознесения в Коломенском
* Ансамбль Новодевичьего монастыря

### Набор данных по ОКН находится на портале открытых данных правительства Москвы
Набор данных "Объекты культурного наследия города Москвы" содержит информацию об объектах, находящихся под государственной охраной, а именно: объектах культурного наследия, включенных в единый государственный реестр объектов культурного наследия (памятников истории и культуры) народов Российской Федерации и выявленных объектах культурного наследия.
#### Описание
https://data.mos.ru/opendata/7705021556-obekty-kulturnogo-naslediya-i-vyyavlennye-obekty-kulturnogo-naslediya/description?versionNumber=6&releaseNumber=19
#### Папорт
https://data.mos.ru/opendata/7705021556-obekty-kulturnogo-naslediya-i-vyyavlennye-obekty-kulturnogo-naslediya/passport?versionNumber=6&releaseNumber=19
#### Вход для разработчиков для получения ключа и чтения документации
https://apidata.mos.ru/
#### Структура данных
https://data.mos.ru/apiproxy/opendata/7705021556-obekty-kulturnogo-naslediya-i-vyyavlennye-obekty-kulturnogo-naslediya/structure-20200310(vs6).json

### Данные по стоимости недвижимости в районах

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes
# !conda install -c conda-forge geopy --yes
# !conda update -n base -c defaults conda
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows',500)
import json
import requests #
from pandas import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import folium # map rendering library
import lxml
import time
print('Done!')

Solving environment: ...working... done

# All requested packages already installed.

Done!


In [238]:
url='https://data.mos.ru/apiproxy/opendata/7705021556-obekty-kulturnogo-naslediya-i-vyyavlennye-obekty-kulturnogo-naslediya/structure-20200310(vs6).json'
results = requests.get(url).json()

In [239]:
print(results)
# df_desc = json_normalize(results)

{'Id': 530, 'IdentificationNumber': '7705021556-Obektykulturnogonaslediya', 'CategoryId': 5, 'CategoryCaption': 'Культура', 'DepartmentId': 7, 'DepartmentCaption': 'Департамент культурного наследия города Москвы', 'Caption': 'Объекты культурного наследия', 'Description': 'Объекты культурного наследия и выявленные объекты культурного наследия, расположенные в пределах установленных границ города Москвы.', 'FullDescription': '<p class="MsoNormal"><span style="font-size:9.0pt;\nfont-family:&quot;Tahoma&quot;,&quot;sans-serif&quot;">Набор данных "Объекты культурного\nнаследия города Москвы" содержит информацию об объектах, находящихся под\nгосударственной охраной, а именно: объектах культурного наследия, включенных в\nединый государственный реестр объектов культурного наследия (памятников истории\nи культуры) народов Российской Федерации и выявленных объектах культурного\nнаследия.<o:p></o:p></span></p><p class="MsoNormal"><span style="font-size:9.0pt;\nfont-family:&quot;Tahoma&quot;,&quot

In [240]:
json_normalize(results,max_level=20)

,Id,IdentificationNumber,CategoryId,CategoryCaption,DepartmentId,DepartmentCaption,Caption,Description,FullDescription,Keywords,ContainsGeodata,VersionNumber,VersionDate,ItemsCount,Columns
0,530,7705021556-Obektykulturnogonaslediya,5,Культура,7,Департамент культурного наследия города Москвы,Объекты культурного наследия,Объекты культурного наследия и выявленные объе...,"<p class=""MsoNormal""><span style=""font-size:9....","культура, культурное наследие, памятники архит...",True,6.19,19.05.2020,8317,"[{'Name': 'ObjectNameOnDoc', 'Caption': 'Наиме..."


In [245]:
url ='https://apidata.mos.ru/version'
results = requests.get(url)
if results.ok:
    print('Version API is :{}'.format(results.json()['Version']))
else:
    print('"Houston, we have a problem!"----->', results)

Version API is :1


In [ ]:
url='https://apidata.mos.ru/v1/datasets/530/rows?api_key=21d82d2344cd3f6e9deba56e6956cffa'
results = requests.get(url)
if results.ok:
    print('What we got: \n{}'.format(results.json()))
else:
    print('"Houston, we have a problem!"----->', results)

In [264]:
results['Id']

TypeError: 'Response' object is not subscriptable